In [1]:
## Step 1: Environment Setup
import arcpy
import arcpy.mp
import random
import requests
import zipfile
import os
import time
import warnings
warnings.filterwarnings("ignore")

# Start time
start_time = time.time()

In [2]:
## Step 2: Data Setup

# Set working directory
arcpy.env.workspace = r"C:\Users\benla\Desktop\Grad_School\Classes\GIS5571_SpatialDataScience\Final_Project\Data"
print(arcpy.env.workspace)

# Set defautlt gdb path
gdb_path =  r"C:\Users\benla\Desktop\Grad_School\Classes\GIS5571_SpatialDataScience\Final_Project\FinalProject_aprx\Default.gdb"

C:\Users\benla\Desktop\Grad_School\Classes\GIS5571_SpatialDataScience\Final_Project\Data


In [10]:
## Step 2: Data Setup

# Get request to download Hennepin County parcels dataset

# Define the download path
parcels_path = os.path.join(arcpy.env.workspace, "County_Parcels.zip")

# Make the get request
response = requests.get("https://hub.arcgis.com/api/v3/datasets/7975aabf6e1e42998a40a4b085ffefdf_1/downloads/data?format=shp&spatialRefId=26915&where=1%3D1")

# Write the response content to a file
with open(parcels_path, 'wb') as file:
    file.write(response.content)
print("File downloaded successfully")

# Unzip the file
with zipfile.ZipFile(parcels_path, 'r') as zip_ref:
    zip_ref.extractall(arcpy.env.workspace)
print("File unzipped successfully")

File downloaded successfully
File unzipped successfully


In [11]:
## Step 2: Data Setup

# Get request to download census tracts shapefile for 7-county metro area

# Define the download path
census_tracts_path = os.path.join(arcpy.env.workspace, "Census_Tracts.zip")

# Make the get request
response = requests.get("https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/society_census2020tiger/shp_society_census2020tiger.zip")

# Write the response content to a file
with open(census_tracts_path, 'wb') as file:
    file.write(response.content)
print("File downloaded successfully")

# Unzip the file
with zipfile.ZipFile(census_tracts_path, 'r') as zip_ref:
    zip_ref.extractall(arcpy.env.workspace)
print("File unzipped successfully")

File downloaded successfully
File unzipped successfully


In [12]:
## Step 2: Data Setup

# Get request to download pollution source point data from My Neighborhood Sites (MPCA)

# Define the download path
pollution_data_path = os.path.join(arcpy.env.workspace, "Pollution_Data.zip")

# Make the get request
response = requests.get("https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_pca/env_my_neighborhood/shp_env_my_neighborhood.zip")

# Write the response content to a file
with open(pollution_data_path, 'wb') as file:
    file.write(response.content)
print("File downloaded successfully")

# Unzip the file
with zipfile.ZipFile(pollution_data_path, 'r') as zip_ref:
    zip_ref.extractall(arcpy.env.workspace)
print("File unzipped successfully")

File downloaded successfully
File unzipped successfully


In [13]:
## Step 2: Data Setup

# Get request to download Hennepin County poverty dataset from MN Department of Health

# Define the download path
poverty_data_path = os.path.join(arcpy.env.workspace, "Poverty_Data.zip")

# Make the get request
response = requests.get("https://mndatamaps.web.health.state.mn.us/interactive/data/zip/PovertyTractData.zip")

# Write the response content to a file
with open(poverty_data_path, 'wb') as file:
    file.write(response.content)
print("File downloaded successfully")

# Unzip the file
with zipfile.ZipFile(poverty_data_path, 'r') as zip_ref:
    zip_ref.extractall(arcpy.env.workspace)
print("File unzipped successfully")

File downloaded successfully
File unzipped successfully


In [14]:
## Step 2: Data Setup

# Read input datasets
parcels_full = r"County_Parcels.shp"
pollution_sources_full = r"my_neighborhood_sites.shp"
census_tracts = r"Census2020TigerTract.shp"
poverty_csv = r"./PovertyTractData/PovertyTract2019.csv"
demographics_csv = r"CensusTract_Demographic_Data.csv"
GEMS_lab_data = r"full_parcels_ranked_final_v1.shp"

In [15]:
## Step 2: Data Setup

# Create feature layers
arcpy.MakeFeatureLayer_management(parcels_full, "parcels_layer")

# Get the extent of the parcels layer
extent = arcpy.Describe("parcels_layer").extent

# Clip pollution source points to the extent of parcels
arcpy.analysis.Clip(
    in_features=pollution_sources_full,
    clip_features="parcels_layer",
    out_feature_class="pollution_source_points_hennepin"
)

<Result 'C:\\Users\\benla\\Desktop\\Grad_School\\Classes\\GIS5571_SpatialDataScience\\Final_Project\\Data\\pollution_source_points_hennepin.shp'>

In [16]:
## Step 3: Data Analysis

## Count pollution source points within each parcel, if no points within parcel, calculate distance to nearest pollution point 
# Perform Spatial Join to count points within each polygon
output_layer="parcels_with_pollution_count"
arcpy.analysis.SpatialJoin(
    target_features= "parcels_layer",
    join_features="pollution_source_points_hennepin",
    out_feature_class=output_layer,
    join_type="KEEP_ALL",
    match_option="CONTAINS",
    field_mapping=arcpy.FieldMappings()
)

# Add a new field for point count and another for nearest distance IF no points within parcel
arcpy.management.AddField(output_layer, "pts_in", "LONG")
arcpy.management.AddField(output_layer, "dist_near", "DOUBLE")

# Calculate the count of points within each polygon
arcpy.management.CalculateField(
    in_table=output_layer,
    field="pts_in",
    expression="!Join_Count!"
)

<Result 'parcels_with_pollution_count'>

In [17]:
## Step 3: Data Analysis

# Select polygons with zero points
zero_points_layer = "zero_points_layer"
arcpy.management.MakeFeatureLayer(output_layer, zero_points_layer, "pts_in = 0")

# Calculate the nearest distance for polygons with zero points
arcpy.analysis.Near(
    in_features=zero_points_layer,
    near_features="pollution_source_points_hennepin",
    search_radius="",
    location="NO_LOCATION",
    angle="NO_ANGLE",
    method="PLANAR"
)

# Update the main output layer with nearest distance
with arcpy.da.UpdateCursor(output_layer, ["pts_in", "NEAR_DIST", "dist_near"]) as cursor:
    for row in cursor:
        if row[0] == 0:  # If pts_in is 0
            row[2] = row[1]  # Set dist_near to NEAR_DIST value
        else:
            row[2] = 0.0  # Set dist_near to 0.0 if there are points within the polygon
        cursor.updateRow(row)

# Verify the result
result_count = int(arcpy.GetCount_management(output_layer).getOutput(0))
print(f"Number of parcels processed: {result_count}")

Number of parcels processed: 446534


In [18]:
## Step 3: Data Analysis

## Clean parcel + pollution data
# List of fields to keep
fields_to_keep = ['FID', 'Shape', 'OBJECTID', 'PID', 'LAT', 'LON', 'ShapeSTAre', 'ShapeSTLen', 'pts_in', 'dist_near']

# List all fields in the output layer
all_fields = [f.name for f in arcpy.ListFields(output_layer)]

# Identify fields to delete (fields not in the keep list)
fields_to_delete = [f for f in all_fields if f not in fields_to_keep]

# Delete unwanted fields
if fields_to_delete:
    arcpy.management.DeleteField(output_layer, fields_to_delete)

# Delete the zero_points_layer and original parcels_sample from the map
if arcpy.Exists("zero_points_layer"):
    arcpy.management.Delete("zero_points_layer")
if arcpy.Exists("parcels_sample"):
    arcpy.management.Delete("parcels_sample")

print(f"Unwanted fields removed, 'zero_points_layer' and 'parcels_sample' deleted.")

Unwanted fields removed, 'zero_points_layer' and 'parcels_sample' deleted.


In [19]:
## Step 3: Data Analysis

## Bring in poverty data and simultaneously filter it for Hennepin County and clean the data
# Define the path to the CSV file and the output table name
pov_output_table = "Poverty_data_table"

# Convert CSV to Arc Table 
arcpy.TableToTable_conversion(poverty_csv, gdb_path, pov_output_table)

# Create new table for Hennepin County poverty data only
hennepin_pov_table = "Hennepin_pov_data"
hennepin_pov_table_path = os.path.join(gdb_path, hennepin_pov_table)
arcpy.CreateTable_management(gdb_path, hennepin_pov_table)

# Add fields to Hennepin poverty data table
arcpy.AddField_management(hennepin_pov_table_path, "povdata_FIPS", "TEXT")
arcpy.AddField_management(hennepin_pov_table_path, "County", "TEXT")
arcpy.AddField_management(hennepin_pov_table_path, "Num_in_Pov", "LONG")
arcpy.AddField_management(hennepin_pov_table_path, "Pct_in_Pov", "DOUBLE")
arcpy.AddField_management(hennepin_pov_table_path, "State_Comp", "TEXT")

# Use a search cursor to filter rows for 'Hennepin' and an insert cursor to add the data to the new table
with arcpy.da.SearchCursor("Poverty_data_table", ['FIPS', 'PrimaryCounty', 'AllAgesCount', 'AllAgesPct', 'AllAgesSigText']) as search_cursor:
    with arcpy.da.InsertCursor(hennepin_pov_table_path, ['povdata_FIPS', 'County','Num_in_Pov', 'Pct_in_Pov', 'State_Comp']) as insert_cursor:
        for row in search_cursor:
            # Check if the row is from Hennepin County
            if row[1] == 'Hennepin':
                # Insert the filtered and renamed data into the new table
                insert_cursor.insertRow((row[0], row[1], row[2], row[3], row[4]))

if arcpy.Exists("Poverty_data_table"):
    arcpy.management.Delete("Poverty_data_table")

print("The filtered and renamed data has been successfully written to the new table, original table deleted.")

The filtered and renamed data has been successfully written to the new table, original table deleted.


In [20]:
## Step 3: Data Analysis

## Bring in demographics data
# Define the path to the CSV file and the output table name
dem_output_table = "Demographics_data_table"

# Convert CSV to Arc Table 
arcpy.TableToTable_conversion(demographics_csv, gdb_path, dem_output_table)

# Ensure 'GEO_ID' is text and properly formatted
arcpy.management.AddField(dem_output_table, "GEO_ID_TEXT", "TEXT")
arcpy.management.CalculateField(dem_output_table, "GEO_ID_TEXT", "str(!GEO_ID!).zfill(11)")

<Result 'Demographics_data_table'>

In [21]:
## Step 3: Data Analysis

## Join poverty data to census tracts to give it geographic context
arcpy.management.JoinField(census_tracts, "GEOID20", 
                           "Hennepin_pov_data", "povdata_FIPS", 
                           ["Num_in_Pov", "Pct_in_Pov", "State_Comp"])

print("The join with poverty data was successful.")

## Join demographics data to census tracts to give it geographic context
arcpy.management.JoinField(census_tracts, "GEOID20", 
                           "Demographics_data_table", "GEO_ID_TEXT", 
                           ["Per_White", "Per_Minor"])

print("The join with demographics data was successful.")

The join with poverty data was successful.
The join with demographics data was successful.


In [22]:
## Step 3: Data Analysis

# Perform Spatial Join Between Census Tracts and Parcels
parcels_pollution_poverty_race = os.path.join(gdb_path, "parcels_pollution_poverty_race")
arcpy.analysis.SpatialJoin("parcels_with_pollution_count", "Census2020TigerTract", parcels_pollution_poverty_race, 
                           join_type="KEEP_ALL", match_option="WITHIN")

# Aggregate the Data by PID
aggregated_table = os.path.join(gdb_path, "aggregated_parcels_pollution_poverty_race")
arcpy.analysis.Statistics(parcels_pollution_poverty_race, aggregated_table,
                          [["Pct_in_Pov", "MAX"], ["Num_in_Pov", "MAX"], ["State_Comp", "FIRST"], ["Per_Minor", "MAX"]],
                          "PID")

# Join the Aggregated Data Back to the Parcels
parcels_updated = os.path.join(gdb_path, "parcels_updated")
arcpy.management.JoinField(parcels_pollution_poverty_race, "PID", aggregated_table, "PID",
                           ["MAX_Pct_in_Pov", "MAX_Num_in_Pov", "FIRST_State_Comp", "MAX_Per_Minor"])

# Save the final output as a new layer
arcpy.management.CopyFeatures(parcels_pollution_poverty_race, parcels_updated)

# Delete the proginal parcels_pollution_poverty layer from the map
if arcpy.Exists("parcels_pollution_poverty_race"):
    arcpy.management.Delete("parcels_pollution_poverty_race")

print("Spatial join and aggregation completed successfully.")

Spatial join and aggregation completed successfully.


In [23]:
## Step 3: Data Analysis

# Clean parcel data

# List of fields to keep
fields_to_keep = ['FID', 'OBJECTID_1','Shape_Area', 'Shape_Length','Shape', 'OBJECTID', 'PID', 'LAT', 'LON', 'ShapeSTAre', 'ShapeSTLen', 'pts_in', 'dist_near', 'MAX_Pct_in_Pov', 'MAX_Per_Minor']

# Get the fields from the parcels_updated layer
fields_in_table = arcpy.ListFields("parcels_updated")

# Identify fields to delete
fields_to_delete = [field.name for field in fields_in_table if field.name not in fields_to_keep]

# Delete the unwanted fields
if fields_to_delete:
    arcpy.management.DeleteField("parcels_updated", fields_to_delete)

print("Cleaning completed successfully.")

Cleaning completed successfully.


In [24]:
## Step 4: Calculate Scores and Rank Parcels 

# Define pollution scoring function

# Calculate the maximum values for 'pts_in' and 'dist_near'
max_within = None
max_distance = None

with arcpy.da.SearchCursor(parcels_updated, ["pts_in", "dist_near"]) as cursor:
    for row in cursor:
        if row[0] is not None:
            max_within = max(max_within, row[0]) if max_within is not None else row[0]
        if row[1] is not None:
            max_distance = max(max_distance, row[1]) if max_distance is not None else row[1]

# Define the scoring function
def calculate_pollution_score(pts_in, dist_near):
    if pts_in is None or dist_near is None:
        return 0
    if pts_in > 0:
        return 5 + (pts_in / max_within) * 5
    else:
        distance_score = 5 - (dist_near / max_distance * 5)
        return max(distance_score, 0)

# Add a new field for the pollution score
pollution_score_field = "pollution_score"
if not arcpy.ListFields(parcels_updated, pollution_score_field):
    arcpy.management.AddField(parcels_updated, pollution_score_field, "DOUBLE")

# Calculate and update the pollution score for each parcel
with arcpy.da.UpdateCursor(parcels_updated, ["pts_in", "dist_near", pollution_score_field]) as cursor:
    for row in cursor:
        row[2] = round(calculate_pollution_score(row[0], row[1]), 2)
        cursor.updateRow(row)

print("Pollution scores calculated and updated successfully.")

Pollution scores calculated and updated successfully.


In [25]:
## Step 4: Calculate Scores and Rank Parcels 

# Define poverty scoring function

# Define the scoring function
def calculate_poverty_score(MAX_Pct_in_Pov):
    if MAX_Pct_in_Pov is None:
        return 0
    return (MAX_Pct_in_Pov / 100) * 10

# Add a new field for the poverty score
poverty_score_field = "poverty_score"
if not arcpy.ListFields(parcels_updated, poverty_score_field):
    arcpy.management.AddField(parcels_updated, poverty_score_field, "DOUBLE")

# Calculate and update the poverty score for each parcel
with arcpy.da.UpdateCursor(parcels_updated, ["MAX_Pct_in_Pov", poverty_score_field]) as cursor:
    for row in cursor:
        row[1] = round(calculate_poverty_score(row[0]), 2)
        cursor.updateRow(row)

print("Poverty scores calculated and updated successfully.")

Poverty scores calculated and updated successfully.


In [26]:
## Step 4: Calculate Scores and Rank Parcels 

# Define demographics scoring function

# Define the scoring function
def calculate_demographics_score(MAX_Per_Minor):
    if MAX_Per_Minor is None:
        return 0
    return (MAX_Per_Minor / 100) * 10

# Add a new field for the demographics score
demographics_score_field = "demographics_score"
if not arcpy.ListFields(parcels_updated, demographics_score_field):
    arcpy.management.AddField(parcels_updated, demographics_score_field, "DOUBLE")

# Calculate and update the demographics score for each parcel
with arcpy.da.UpdateCursor(parcels_updated, ["MAX_Per_Minor", demographics_score_field]) as cursor:
    for row in cursor:
        row[1] = round(calculate_demographics_score(row[0]), 2)
        cursor.updateRow(row)

print("Demographics scores calculated and updated successfully.")

Demographics scores calculated and updated successfully.


In [27]:
## Step 4: Calculate Scores and Rank Parcels 

# Original Weighting criteria
pollution_weight = 0.50
poverty_weight = 0.25
demographics_weight = 0.25


## Overall Score Calculation

# Add a new field for the composite score
composite_score_field = "composite_score"
if not arcpy.ListFields(parcels_updated, composite_score_field):
    arcpy.management.AddField(parcels_updated, composite_score_field, "DOUBLE")

# Calculate and update the composite score for each parcel
with arcpy.da.UpdateCursor(parcels_updated, ["pollution_score", "poverty_score", "demographics_score", composite_score_field]) as cursor:
    for row in cursor:
        pollution_score = row[0] if row[0] is not None else 0
        poverty_score = row[1] if row[1] is not None else 0
        demographics_score = row[2] if row[2] is not None else 0
        composite_score = ((pollution_score * pollution_weight) +
                           (poverty_score * poverty_weight) +
                           (demographics_score * demographics_weight))
        row[3] = round(composite_score, 2)
        cursor.updateRow(row)

print("Composite scores calculated and updated successfully.")

Composite scores calculated and updated successfully.


In [28]:
## Step 4: Calculate Scores and Rank Parcels

# Add a new field for rank
rank_field = "rank"
if not arcpy.ListFields(parcels_updated, rank_field):
    arcpy.management.AddField(parcels_updated, rank_field, "LONG")

# Create a list to store composite scores and their OBJECTIDs
scores = []

# Collect composite scores and OBJECTIDs
with arcpy.da.SearchCursor(parcels_updated, ["OBJECTID", "composite_score"]) as cursor:
    for row in cursor:
        scores.append((row[0], row[1]))

# Sort the list by composite scores in descending order
scores.sort(key=lambda x: x[1], reverse=True)

# Assign ranks
rank_dict = {obj_id: rank + 1 for rank, (obj_id, score) in enumerate(scores)}

# Update the rank field in the feature class
with arcpy.da.UpdateCursor(parcels_updated, ["OBJECTID", rank_field]) as cursor:
    for row in cursor:
        row[1] = rank_dict[row[0]]
        cursor.updateRow(row)

print("Ranking completed successfully.")

Ranking completed successfully.


In [29]:
## Step 5: Sensitivity Analysis 

# Define the weight schemes of the sensitivity analysis
weight_schemes = {
    "Scheme_1": {"pollution": 0.50, "poverty": 0.25, "demographics": 0.25}, # Original weighting scheme
    "Scheme_2": {"pollution": 0.34, "poverty": 0.33, "demographics": 0.33}, # Nearly equal weighting scheme to all three variables (pollution given 0.01 heavier weight to sum to 1)
    "Scheme_3": {"pollution": 0.10, "poverty": 0.45, "demographics": 0.45}, # Pollution only weighted at 0.1, giving almost all (0.9) priority evenly to poverty and demographics
    "Scheme_4": {"pollution": 0.15, "poverty": 0.15, "demographics": 0.70}, # Demographics weighted at 0.5 
    "Scheme_5": {"pollution": 0.15, "poverty": 0.70, "demographics": 0.15}, # Poverty weighted at 0.5
}

# Add new fields for each composite score scheme
for scheme in weight_schemes.keys():
    composite_field = f"composite_{scheme}"
    if not arcpy.ListFields(parcels_updated, composite_field):
        arcpy.management.AddField(parcels_updated, composite_field, "DOUBLE")

# Recalculate composite scores for each scheme and update the feature class
for scheme, weights in weight_schemes.items():
    composite_field = f"composite_{scheme}"
    pollution_weight = weights["pollution"]
    poverty_weight = weights["poverty"]
    demographics_weight = weights["demographics"]

    with arcpy.da.UpdateCursor(parcels_updated, ["pollution_score", "poverty_score", "demographics_score", composite_field]) as cursor:
        for row in cursor:
            pollution_score = row[0] if row[0] is not None else 0
            poverty_score = row[1] if row[1] is not None else 0
            demographics_score = row[2] if row[2] is not None else 0
            composite_score = (pollution_score * pollution_weight +
                               poverty_score * poverty_weight +
                               demographics_score * demographics_weight)
            row[3] = round(composite_score, 2)
            cursor.updateRow(row)

print("Sensitivity analysis completed. Composite scores for different schemes have been calculated and updated.")

Sensitivity analysis completed. Composite scores for different schemes have been calculated and updated.


In [31]:
## Step 6: Calculate total time

end_time = time.time()
total_time = end_time - start_time
hours = int(total_time // 3600)
minutes = int((total_time % 3600) // 60)
seconds = total_time % 60
print(f"Total time: {hours} hours, {minutes} minutes, {seconds:.2f} seconds")

Total time: 1 hours, 1 minutes, 19.41 seconds
